In [1]:
import numpy as np 
from utils_cnn import padding

import os 
import sys 

import scipy.signal as signal

import torch 
import torch.functional as F

In [2]:
"""
Define test case: 
    1. One input input of size (1, 32, 32) 
    2. 
"""

# Generate test image. 
input_image = np.random.rand(3, 32, 32)
o1 = np.random.rand(6, 28, 28)



In [3]:
def MaxPool( conv_image , kernel_size , stride=1): 
        """
        Subsampling layer. 

        Parameters:
        -----------
            - conv_image : image in numpy and dimension (C, H, W) 
            - kernel_size : kernel_size of the pooling layer. The kernel stays rectangular  
            - stride : stide of the pooling layer 

        Returns:
        --------
        pooled in numpy 
        
        """
        # Get input dimensions 
        input_channel , input_height , input_width = conv_image.shape
    
        # Get the output dimensions 
        output_height = (input_height - kernel_size) // stride+1 
        output_width = (input_width - kernel_size) // stride+1 

        print("input height : " , input_height ) 
        print("input width : " , input_width ) 
        print("output_height : ", output_height )
        print("output_width : ", output_width ) 
        
        
        # initialize a np 3D array for pooling kernel 
        pooled_ = np.zeros((input_channel, output_height, output_width )) 
        #print(pooled_.shape) 

        #sys.exit(-1) 
            
        # 2. Scan through the input image and get the max 

        # The iteration below will be honsistent for 3 dimensions; as of now it is for 2 . 
        # Apply pooling operation per channel
    
        for c in range(input_channel): # iterate across the channel 
            
            for j in range(0, input_height - kernel_size + 1, stride): # iterate across the height
                
                for i in range(0, input_width - kernel_size + 1, stride): # iterate across the width 
                    
                    # Define the current window
                    
                    window = conv_image[c, j:j + kernel_size, i:i + kernel_size]
        
                    # Get the max value in the window
                    max_value = np.max(window)
        
                    # Map max value to the output matrix
                    pooled_[c, j // stride, i // stride] = max_value # fixed to get integer size
        
        return pooled_ 

pooled_ = MaxPool(o1, 2,stride=2)

print(pooled_.shape) 
sys.exit(0) 

input height :  28
input width :  28
output_height :  14
output_width :  14
(6, 14, 14)


SystemExit: 0

/Users/shahriar/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
def generate_cnn_patches(image):
        num_channel, image_h, image_w = image.shape

        if isinstance(self.kernel_size, int):
            kernel_h = kernel_w = self.kernel_size
        elif isinstance(self.kernel_size, tuple):
            kernel_h, kernel_w = self.kernel_size


        num_channel, image_h, image_w = image.shape
        # Initialize storage for patches
        patches = []
        for channel in range(num_channel):
            channel_patch = []
            for i in range(0, image_h - kernel_h + 1, self.stride):
                for j in range(0, image_w - kernel_w + 1, self.stride):
                    patch = image[channel, i:i+kernel_h, j:j+kernel_w]
                    channel_patch.append(patch)
            channel_patch = np.array(channel_patch)
            patches.append(channel_patch)
        patches = np.array(patches)

        return patches
    
def convolute(patch, kernel):
    conv_mat = signal.correlate2d(patch, kernel, mode='valid')
    return conv_mat
    

In [13]:
def convolve(img, kernel):
        patches = self.generate_cnn_patches(img)
        _, _, output_h, output_w = self.output_shape
        
        convoluted_output_shape = (self.out_channel, output_h, output_w)
        convoluted_output = np.zeros((convoluted_output_shape))

        for i in range(self.out_channel):
            for j in range(self.in_channel):
                row, col = 0, 0
                for patch in patches[j]:
                    convoluted_output[i][row][col] = self.convolute(patch, kernel[i][j])
                    if col >= output_w - 1:
                        col = 0
                        row += 1
                    else:
                        col += 1
            convoluted_output[i] += self.biases[i]
        
        return convoluted_output
    

In [5]:
###

In [18]:
"""

- generate dummy dL_dz 
- generate dummy output 


"""

dL_dz_test = np.random.rand(6, 28, 28)
z = np.random.rand( 6, 28, 28) 

kernel = np.random.rand(6,5, 5)




def backward_wrt_input(input_image, kernel ,stride=1):     
    
    """
    Description: 
    -----------
    The image input is an input of dimension (1, H, W). 
    KEY INSIGHT : The CNN Backpropagation operation with stride>1 is identical to a stride = 1. 
    

    Parameters: 
    -----------

    Returns: 
    --------
    updated weights 

    ----
    
    Note: 
        i)  See if stride is a factor here. 
        ii) 
        iii) 
        
        key formula : dL_dx = full_convolution(kernel, loss gradient dL_dz ) 
        dL_dO is dL_dz in MMP's code

        z/O is the output of the convolution layer 
        dL_do/dL_dz 
    
    ----
    """    
    
    print(input_image.shape)   
    # Since this is a single layer, let us ignore the dimension C.
    
    # Step 1. filp the filter/kernel 180 degree; assume that a single kernel is 3D i.e. (1, H, W) 
    kernel_rotated = kernel[: , ::-1, ::-1 ] # replace this with
    
    # Step 2. pad the dL_dz
    dL_dz_padded_1 = padding(dL_dz_test, 1) # padding dL_dZ with 1) 

    """
    
    dL_dz_test_padded_1 shape : (1, 30, 30) 
    kernel_rotated : (1, 5, 5) 
    
    
    convolutional arithmetic : ( ((30-5) + 2* padding) / stride) ) + 1  
    
    
    """

    image_height = dL_dz_padded_1.shape[-1]
    kernel_height = kernel_rotated.shape[-1]
    stride_ = 1  

    output_dimension = image_height - kernel_height // 1

    print("output dimension : ", output_dimension)

    print("---------") 
    
    print("dL_dz before padding : " , dL_dz_test.shape ) 
    print("dL_dz_padded_1 , after initialization : ", dL_dz_padded_1.shape ) 

    print("Update : Padding applied successfully" ) 
    output = np.zeros( (output_dimension, output_dimension) ) 

    print("output shape ", output.shape) 
    #sys.exit(-2) 
    
    # Step 3: 
    # perform convolution between dL_dz_padded_1 and kernel_rotated- the results will be in output matrix 

    print("--------------") 
    # Step 3: Perform the convolution operation --->> This should be a separate function., 
    
    # add another loop to iterate across all the channels 
    # (Batch Size, Channel, Height, Width) --> in my code (Channel (C), Height (i), Width (j)) 
    
    for i in range(0, output_dimension):
        
        for j in range(0, output_dimension):
            
            # Extract the patch of the same size as the kernel
            patch = dL_dz_padded_1[0, i:i+kernel_height, j:j+kernel_height]

            # Perform element-wise multiplication and sum
            output[i, j] = np.sum(patch * kernel_rotated[0])

            print("i, j: (", i, j , ")")

    
    #print("dL_dx: After convolution is applied")
    #print(output)
    print("--------------") 


    """
    output is dL/dX 
    """

    
    
    
    
    #for i in range(dl_dz_padded_1.shape[0]): #6, 32, 32 #x = 32, 32     
    #    convolve(dl_dz_padded[i], kernel)
                
                        
        #    output[] = convolute(patch, kernel)

    #print("dL_dx : after convolution is applied", ) 

    print("output " , output.shape) 

    # update the weight 
         

backward_wrt_input(input_image, kernel, stride=1)    

(3, 32, 32)
output dimension :  25
---------
dL_dz before padding :  (6, 28, 28)
dL_dz_padded_1 , after initialization :  (6, 30, 30)
Update : Padding applied successfully
output shape  (25, 25)
--------------
i, j: ( 0 0 )
i, j: ( 0 1 )
i, j: ( 0 2 )
i, j: ( 0 3 )
i, j: ( 0 4 )
i, j: ( 0 5 )
i, j: ( 0 6 )
i, j: ( 0 7 )
i, j: ( 0 8 )
i, j: ( 0 9 )
i, j: ( 0 10 )
i, j: ( 0 11 )
i, j: ( 0 12 )
i, j: ( 0 13 )
i, j: ( 0 14 )
i, j: ( 0 15 )
i, j: ( 0 16 )
i, j: ( 0 17 )
i, j: ( 0 18 )
i, j: ( 0 19 )
i, j: ( 0 20 )
i, j: ( 0 21 )
i, j: ( 0 22 )
i, j: ( 0 23 )
i, j: ( 0 24 )
i, j: ( 1 0 )
i, j: ( 1 1 )
i, j: ( 1 2 )
i, j: ( 1 3 )
i, j: ( 1 4 )
i, j: ( 1 5 )
i, j: ( 1 6 )
i, j: ( 1 7 )
i, j: ( 1 8 )
i, j: ( 1 9 )
i, j: ( 1 10 )
i, j: ( 1 11 )
i, j: ( 1 12 )
i, j: ( 1 13 )
i, j: ( 1 14 )
i, j: ( 1 15 )
i, j: ( 1 16 )
i, j: ( 1 17 )
i, j: ( 1 18 )
i, j: ( 1 19 )
i, j: ( 1 20 )
i, j: ( 1 21 )
i, j: ( 1 22 )
i, j: ( 1 23 )
i, j: ( 1 24 )
i, j: ( 2 0 )
i, j: ( 2 1 )
i, j: ( 2 2 )
i, j: ( 2 3 )
i, j

In [ ]:
a_ = padding(image, 2)

print(a_.shape) 